* Hyperparameter tuning of the 5 classifiers for emotional state detection
* 5 fold cross validation with grid-search
* Multiclass classification

In [1]:
import pandas as pd
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from pprint import pprint
from sklearn.model_selection import train_test_split

from sklearn import metrics   
from sklearn.feature_selection import SelectFromModel,RFECV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, PredefinedSplit
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler


from sklearn import metrics   
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SVMSMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

pd.options.mode.chained_assignment = None
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.filterwarnings('always')
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from imblearn.metrics import specificity_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix
from sklearn import metrics   

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

from catboost import CatBoostClassifier, Pool, cv
from sklearn.neural_network import MLPClassifier


#from pandas_ml import ConfusionMatrix

#import collections

/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.ba

In [2]:
def read_input(p):
    #Read input file of each person
    filename='data/NonOverlap_w5_emoChange_SelFeat_data_p'+str(p)+'.csv'
    raw_df= pd.read_csv(filename)
    
    #drop all variables that contain all NANs
    raw_df.dropna(axis=1,how='all', inplace=True)
    #reset the index
    raw_df.reset_index(drop=True, inplace=True)
    #drop columns with all zeros in pandas dataframe
    raw_df=raw_df.T[(raw_df!=0).any()].T
    
    #print("The shape of the dataframe is ",raw_df.shape)
    #print(raw_df['emotion'].value_counts())
    return raw_df

In [3]:
# replace NANs with -999
def prep_data(data):
    return data.fillna(-999)

In [4]:
#drop columns
def drop_cols(data, col_list):
    return data.drop(col_list, axis=1)

In [5]:
# normalize data with minmax
def scale_data(trn_x, tst_x):
    
    sc= StandardScaler()
    scaled_trn_x = sc.fit_transform(trn_x)
    scaled_tst_x = sc.fit_transform(tst_x)
    
    return scaled_trn_x, scaled_tst_x

In [6]:
# oversampling with SMOTE with 'minority' and 'not majority'
def over_sample_SMOTE(X_train, y_train):
    sm=SMOTE(sampling_strategy='not majority', random_state=10) # 'minority'
    X_train_ovr, y_train_ovr=sm.fit_sample(X_train, y_train)

    #print(X_train_ovr.shape, y_train_ovr.shape)
    return X_train_ovr, y_train_ovr

In [7]:
# oversampling with SVMSMOTE 
def over_sample_SVMSMOTE(X_train, y_train):
    sm=SVMSMOTE(random_state=10)
    
    X_train_ovr, y_train_ovr=sm.fit_sample(X_train, y_train)

    #print(X_train_ovr.shape, y_train_ovr.shape)
    return X_train_ovr, y_train_ovr

In [8]:
def select_k_features(X_train_scaled,X_test_scaled,y_train,k):
    selection = SelectKBest(mutual_info_classif, k)
    X_train = selection.fit_transform(X_train_scaled,y_train)
    X_test = selection.transform(X_test_scaled)
    
    return X_train, X_test

In [9]:
# define random state to re-generate the same result
random_state = 43

# total persons
p_list=[8, 10,12,13,15,20,21,25, 27, 33,35,40,46,48,49,52,54,55]
#p_list=[8]

# #of folds
n_fold=5

In [10]:
def print_results(accu, bl_accu, prec, rec_, spec_, roc_, f1_):   
    print('.....................')
    print("Average Accuracy: %.2f%% (%.2f)" % (np.mean(accu), np.std(accu)))
    print("Average Balanced_accuracy: %.2f%% (%.2f)" % (np.mean(bl_accu),np.std(bl_accu)))
    print("Average Precision: %.2f%% (%.2f)" % (np.mean(prec),np.std(prec)))
    print("Average Recall: %.2f%% (%.2f)" % (np.mean(rec_),np.std(rec_)))
    print("Average Specificity: %.2f%% (%.2f)" % (np.mean(spec_),np.std(spec_)))
    print("Average ROC AUC: %.2f%% (%.2f)" % (np.mean(roc_),np.std(roc_)))
    print("Average F1 score: %.2f%% (%.2f)" % (np.mean(f1_),np.std(f1_)))
    print('..................................................')
    print('\n')

In [11]:
pipe = Pipeline([('scaler', StandardScaler()), # MinMaxScaler()
                 ('selector', SelectKBest(mutual_info_classif, k=90)), #
                 ('classifier', LogisticRegression())])

search_space = [{'selector__k': [ 50, 70, 90]},
                
                {'classifier': [LogisticRegression(solver='lbfgs')],
                 'classifier__C': [0.01, 0.1, 1.0],
                 'classifier__penalty': ['l1', 'l2', None],
                 'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                 'classifier__max_iter':[100, 150, 200], 
                 'classifier__class_weight':[None, 'balanced']},
                 
                {'classifier': [RandomForestClassifier()],
                 'classifier__max_depth': [5, 10, 30, None],
                 'classifier__criterion':['gini','entropy'], 
                 'classifier__bootstrap': [True],
                 'classifier__max_features':['log2', None],
                 'classifier__n_estimators': [50, 100, 200, 300, 400]},
                
                {'classifier': [MLPClassifier(random_state=1, early_stopping=True)],
                 'classifier__hidden_layer_sizes' : [(50, 50, 50), (50, 100, 50), (20, 20, 20), (30, ), (50,),(100,)], 
                 'classifier__activation' : ['tanh', 'relu', 'logistic'],
                 'classifier__max_iter':[50, 100, 150, 200, 300],
                 'classifier__solver': ['sgd', 'adam', 'lbfgs'],
                 'classifier__alpha': [0.0001, 0.001, 0.05]},
                
                {'classifier': [CatBoostClassifier(random_seed=1)],
                 'classifier__learning_rate': [0.05, 0.1, 0.15, 0.2]},
                
                {'classifier': [XGBClassifier(objective='binary:logistic', random_state=1)],
                 'classifier__learning_rate': [0.05, 0.1, 0.15, 0.2],
                 'classifier__colsample_bytree':[.5, .75, 1],
                 'classifier__max_depth': np.arange(3, 6, 10),
                 'classifier__n_estimators': [50, 100, 200, 300, 400]}]
                  

scorer = make_scorer(f1_score, average = 'binary')

In [12]:
LR_pipe = Pipeline([('scaler', StandardScaler()), # MinMaxScaler()
                 ('selector', SelectKBest(mutual_info_classif, k=90)), #
                 ('classifier', LogisticRegression())])

LR_search_space = [{'selector__k': [ 50, 70, 90, 110]},
                
                {'classifier': [LogisticRegression(solver='lbfgs')],
                 'classifier__C': [0.01, 0.1, 1.0],
                 'classifier__penalty': ['l1', 'l2', None],
                 'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                 'classifier__max_iter':[100, 150, 200], 
                 'classifier__class_weight':[None, 'balanced']}]
                 
################################################################################          

RF_pipe = Pipeline([('scaler', StandardScaler()), # MinMaxScaler()
                 ('selector', SelectKBest(mutual_info_classif, k=90)), #
                 ('classifier', RandomForestClassifier())])

RF_search_space = [{'selector__k': [ 50, 70, 90, 110]},
                
                {'classifier': [RandomForestClassifier()],
                 'classifier__max_depth': [5, 10, 30, None],
                 'classifier__criterion':['gini','entropy'], 
                 'classifier__bootstrap': [True],
                 'classifier__max_features':['log2', None],
                 'classifier__n_estimators': [50, 100, 200, 300, 400]}]
                  
################################################################################

MLP_pipe = Pipeline([('scaler', StandardScaler()), # MinMaxScaler()
                 ('selector', SelectKBest(mutual_info_classif, k=90)), #
                 ('classifier', MLPClassifier(random_state=1, early_stopping=True))])

MLP_search_space = [{'selector__k': [ 50, 70, 90, 110]},
                
                {'classifier': [MLPClassifier(random_state=1, early_stopping=True)],
                 'classifier__hidden_layer_sizes' : [(50, 50, 50), (50, 100, 50), (20, 20, 20), (30, ), (50,),(100,)], 
                 'classifier__activation' : ['tanh', 'relu', 'logistic'],
                 'classifier__max_iter':[50, 100, 150, 200, 300],
                 'classifier__solver': ['sgd', 'adam', 'lbfgs'],
                 'classifier__alpha': [0.0001, 0.001, 0.05]}]

################################################################################

CB_pipe = Pipeline([('scaler', StandardScaler()), # MinMaxScaler()
                 ('selector', SelectKBest(mutual_info_classif, k=90)), #
                 ('classifier', CatBoostClassifier(random_seed=1))])

CB_search_space = [{'selector__k': [ 50, 70, 90, 110]},
                
                {'classifier': [CatBoostClassifier(random_seed=1, verbose=False)],
                 'classifier__learning_rate': [0.05, 0.1, 0.15, 0.2]}]

################################################################################

XGB_pipe = Pipeline([('scaler', StandardScaler()), # MinMaxScaler()
                 ('selector', SelectKBest(mutual_info_classif, k=90)), #
                 ('classifier', XGBClassifier(objective='binary:logistic', random_state=1))])

XGB_search_space = [{'selector__k': [ 50, 70, 90, 110]},
                
                {'classifier': [XGBClassifier(objective='binary:logistic', random_state=1)],
                 'classifier__learning_rate': [0.05, 0.1, 0.15, 0.2],
                 'classifier__colsample_bytree':[.5, .75, 1],
                 'classifier__max_depth': np.arange(3, 6, 10),
                 'classifier__n_estimators': [50, 100, 200, 300, 400]}]


In [13]:
def grid_search_wrapper(pipe = pipe, search_space = search_space, verbose= False,refit_score=scorer):
    """
    fits a GridSearchCV classifiers using refit_score for optimization
    prints classifier performance metrics
    """
    cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    grid_search = GridSearchCV(pipe, search_space, cv=cross_validation, verbose=verbose,  n_jobs = -1) #scoring=scorer, refit=scorer
    
    grid_search.fit(X, y)
    
    return grid_search


In [21]:
p_list=[8, 10,12,13,15,20,21,25, 27, 33,35,40,46,48,49,52,54,55]

best_LR = []
best_RF = []
best_MLP = []
best_XGB = []
best_CB = []

In [22]:

# for each person in the dataset, find the best hyperparamters for the model in given range 
for p in p_list:
    df=read_input(p)
    #df.head()
    df=prep_data(df)
    # remove day_of_month variable if present in data
    if 'day_of_month' in df.columns:
        drop_col=['day_of_month']
        df=drop_cols(df, drop_col)

     #remove classes that have less then 5 samples
    min_c=df['emotion_change'].value_counts()
    if (min_c <= 5).any():
        r_label=min_c[min_c <= 5].index[0]
        df = df.drop(df.index[df.emotion == r_label])
        
    dataset = df   

    y = dataset['emotion_change'].copy()
    X = dataset.loc[:, dataset.columns != 'emotion_change'].copy()
    #X = X.apply(pd.to_numeric)
    
    #perform grid searc and find the best models 
    print("Person "+str(p))
    print("-------------------------------------------------------")
    pipeline_grid_search_LR = grid_search_wrapper(pipe = LR_pipe, search_space = LR_search_space, verbose=2)
    print(pipeline_grid_search_LR.best_estimator_)
    print(pipeline_grid_search_LR.best_score_)
    best_LR.append(pipeline_grid_search_LR.best_estimator_)
    
    print("-------------------------------------------------------")
    pipeline_grid_search_RF = grid_search_wrapper(pipe = RF_pipe, search_space = RF_search_space, verbose=2)
    print(pipeline_grid_search_RF.best_estimator_)
    print(pipeline_grid_search_RF.best_score_)
    best_RF.append(pipeline_grid_search_RF.best_estimator_)
    
    print("-------------------------------------------------------")
    pipeline_grid_search_MLP = grid_search_wrapper(pipe = MLP_pipe, search_space = MLP_search_space, verbose=2)
    print(pipeline_grid_search_MLP.best_estimator_)
    print(pipeline_grid_search_MLP.best_score_)
    best_MLP.append(pipeline_grid_search_MLP.best_estimator_)
    
    print("-------------------------------------------------------")
    pipeline_grid_search_XGB = grid_search_wrapper(pipe = XGB_pipe, search_space = XGB_search_space, verbose=2)
    print(pipeline_grid_search_XGB.best_estimator_)
    print(pipeline_grid_search_XGB.best_score_)
    best_XGB.append(pipeline_grid_search_XGB.best_estimator_)
    
    print("-------------------------------------------------------")
    pipeline_grid_search_CB = grid_search_wrapper(pipe = CB_pipe, search_space = CB_search_space, verbose=False)
    print(pipeline_grid_search_CB.best_estimator_)
    print(pipeline_grid_search_CB.best_score_)
    best_CB.append(pipeline_grid_search_CB.best_estimator_)
    print("-------------------------------------------------------")


Person 8
-------------------------------------------------------
Fitting 5 folds for each of 274 candidates, totalling 1370 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1370 out of 1370 | elapsed:  3.0min finished


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('selector',
                 SelectKBest(k=90,
                             score_func=<function mutual_info_classif at 0x1a1a1fd8c0>)),
                ('classifier',
                 LogisticRegression(C=0.01, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)
0.940251572327044
-------------------------------------------------------
Fitting 5 folds for each of 84 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed:  1.7min finished


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('selector',
                 SelectKBest(k=90,
                             score_func=<function mutual_info_classif at 0x1a1a1fd8c0>)),
                ('classifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=5, max_features=None,
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                     

...

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('selector',
                 SelectKBest(k=90,
                             score_func=<function mutual_info_classif at 0x1a1a1fd8c0>)),
                ('classifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=5, max_features=None,
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=50, n_jobs=None,
                                      

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 16.8min
[Parallel(n_jobs=-1)]: Done 4070 out of 4070 | elapsed: 16.9min finished


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('selector',
                 SelectKBest(k=90,
                             score_func=<function mutual_info_classif at 0x1a1a1fd8c0>)),
                ('classifier',
                 MLPClassifier(activation='relu', alpha=0.05, batch_size='auto',
                               beta_1=0.9, beta_2=0.999, early_stopping=True,
                               epsilon=1e-08, hidden_layer_sizes=(50, 50, 50),
                               learning_rate='constant',
                               learning_rate_init=0.001, max_fun=15000,
                               max_iter=150, momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=1, shuffle=True, solver='sgd',
                               tol=0.0001, validation_fraction=0.1,
                               

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:  1.8min finished


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('selector',
                 SelectKBest(k=90,
                             score_func=<function mutual_info_classif at 0x1a1a1fd8c0>)),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=0.5, gamma=0,
                               learning_rate=0.15, max_delta_step=0,
                               max_depth=3, min_child_weight=1, missing=None,
                               n_estimators=50, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=1,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
   

In [23]:
best_models = {} # dictionary of best models with best parameters

best_models['Logistic Regression'] = best_LR
best_models['RandomForest Classifier'] = best_RF
best_models['MLP Classifier'] = best_MLP
best_models['XGBoost Classifier'] = best_XGB
best_models['CatBoost Classifier'] = best_CB


In [24]:
#save the best models
np.save('Indiv_best_models.npy', best_models) 

In [ ]:
# Load the best models
best_models = np.load('Indiv_best_models.npy',allow_pickle='TRUE').item()

In [28]:
# this is to get all the detailed performance meterics after selecting the best model parameters
nfolds = 5
skf=StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=random_state)

for model_name, model in best_models.items(): 
    
    accu = []
    prec = []
    rec_ = []
    f1_ = []
    bl_accu = []
    roc_ = []
    spec_ = []

    model_index = -1
    for p in p_list:
        model_index = model_index + 1
        model__ = model[model_index]
        clf = model__['classifier']
        if model_name == 'CatBoost Classifier':
            clf = CatBoostClassifier(random_seed=1, verbose=False,learning_rate= 0.1)

        df=read_input(p)
        #df.head()
        df=prep_data(df)
        # remove day_of_month variable if present in data
        if 'day_of_month' in df.columns:
            drop_col=['day_of_month']
            df=drop_cols(df, drop_col)

         #remove classes that have less then 5 samples
        min_c=df['emotion_change'].value_counts()
        if (min_c <= 5).any():
            r_label=min_c[min_c <= 5].index[0]
            df = df.drop(df.index[df.emotion == r_label])

        dataset = df   

        y = dataset['emotion_change'].copy()
        X = dataset.loc[:, dataset.columns != 'emotion_change'].copy()

        
        avg_ac=0.0
        avg_bl_ac=0.0
        avg_rc=0.0
        avg_pr=0.0
        avg_f1=0.0
        avg_spec=0.0
        avg_roc=0.0
        avg_kp=0.0

        i = 1
        for train_index, test_index in skf.split(X ,y):
            #print("fold", i)
            i+=1

            X_train, X_test = X.iloc[train_index], X.iloc[test_index] 
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            #scale features
            X_train_scaled, X_test_scaled= scale_data(X_train, X_test) 
            #feature selection
            X_train, X_test = select_k_features(X_train_scaled,X_test_scaled,y_train,k=int(str(model__['selector'])[14:16]))

            #oversample training data
            #X_train_imb,y_train_imb=over_sample_SMOTE(X_train, y_train)
            #X_train_imb,y_train_imb=over_sample_SMOTENC(X_train, y_train, index1, index2)
            #X_train_imb,y_train_imb=over_sample_SVMSMOTE(X_train, y_train)


            # train model on imbalance-handled data
            #clf.fit(X_train_imb, y_train_imb)

            #train model on imbalance data 
            clf.fit(X_train, y_train)

            # test model, measure class label and probability score
            y_pred = clf.predict(X_test)
            y_scores = clf.predict_proba(X_test)[:,1]

            #calculate metrices
        
            accuracy = accuracy_score(y_test, y_pred)
            bl_accuracy = balanced_accuracy_score(y_test, y_pred)
            precision=precision_score(y_test, y_pred, labels=np.unique(y_pred))
            recall=recall_score(y_test, y_pred, labels=np.unique(y_pred))
            f1=f1_score(y_test, y_pred, labels=np.unique(y_pred))
            roc=roc_auc_score(y_test, y_scores, labels=np.unique(y_pred))
            spec=specificity_score(y_test, y_pred ,labels=np.unique(y_pred))

            ac=accuracy * 100.0
            pr=precision*100
            rc=recall*100
            f1_p=f1*100
            bl_ac=bl_accuracy*100
            roc=roc*100
            spec=spec*100


            #update average metrices in each fold
            avg_ac+=ac
            avg_bl_ac+=bl_ac
            avg_rc+=rc
            avg_pr+=pr
            avg_f1+=f1_p
            avg_roc+=roc
            avg_spec+=spec
    
        avg_ac = avg_ac/nfolds
        avg_bl_ac = avg_bl_ac/nfolds
        avg_rc = avg_rc/nfolds
        avg_pr = avg_pr/nfolds
        avg_f1 = avg_f1/nfolds
        avg_roc = avg_roc/nfolds
        avg_spec = avg_spec/nfolds
            
        accu.append(avg_ac)
        prec.append(avg_pr)
        rec_.append(avg_rc)
        f1_.append(avg_f1)
        bl_accu.append(avg_bl_ac)
        roc_.append(avg_roc)
        spec_.append(avg_spec)
    
    print('Restuls for: ', model_name)
    print_results(accu, bl_accu, prec, rec_, spec_, roc_, f1_)


/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-

Restuls for:  Logistic Regression
.....................
Average Accuracy: 95.36% (1.95)
Average Balanced_accuracy: 64.23% (9.42)
Average Precision: 50.36% (26.66)
Average Recall: 29.49% (19.26)
Average Specificity: 98.96% (0.93)
Average ROC AUC: 87.31% (13.23)
Average F1 score: 35.12% (20.47)
..................................................




/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-

Restuls for:  RandomForest Classifier
.....................
Average Accuracy: 93.49% (4.73)
Average Balanced_accuracy: 66.88% (10.08)
Average Precision: 49.97% (21.23)
Average Recall: 37.12% (20.78)
Average Specificity: 96.65% (4.91)
Average ROC AUC: 88.01% (5.67)
Average F1 score: 36.40% (17.87)
..................................................




/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-

Restuls for:  MLP Classifier
.....................
Average Accuracy: 94.78% (2.46)
Average Balanced_accuracy: 65.65% (12.36)
Average Precision: 40.60% (26.71)
Average Recall: 33.10% (25.12)
Average Specificity: 98.19% (1.35)
Average ROC AUC: 80.27% (18.56)
Average F1 score: 34.65% (25.38)
..................................................




/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-

Restuls for:  XGBoost Classifier
.....................
Average Accuracy: 94.82% (2.55)
Average Balanced_accuracy: 65.91% (7.93)
Average Precision: 52.34% (17.22)
Average Recall: 33.71% (15.69)
Average Specificity: 98.12% (1.42)
Average ROC AUC: 87.84% (6.81)
Average F1 score: 38.00% (15.92)
..................................................




/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/majed_al-jefri/opt/anaconda3/lib/python3.7/site-

Restuls for:  CatBoost Classifier
.....................
Average Accuracy: 94.29% (3.17)
Average Balanced_accuracy: 66.89% (8.97)
Average Precision: 48.67% (18.09)
Average Recall: 36.45% (18.10)
Average Specificity: 97.34% (2.33)
Average ROC AUC: 87.62% (7.04)
Average F1 score: 38.34% (17.07)
..................................................


